In [2]:
# docker run -p 6333:6333 qdrant/qdrant

In [3]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams
from qdrant_client.http.models import Distance
from pprint import pprint

from embedder import get_sentences_embeddings
from text_splitter import CustomTextSplitter
from retriever import CustomRetriever
from utils import folder_into_db_collection

/home/ilgiz/dev/RAG-On-Obsidian/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
prefixes = {'query': 'query: ', 'instance': 'passage: '}

sentence_embedder = SentenceTransformer('intfloat/multilingual-e5-small')
VECTOR_SIZE = get_sentences_embeddings(
    ['Hello, world!'], sentence_embedder, prefix=prefixes['instance']).shape
collection_name = 'obsidian-vault'

In [5]:
url = "http://localhost:6333"
qdrant_client = QdrantClient(url)

qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=VECTOR_SIZE[1],
        distance=Distance.COSINE),
            )

/tmp/ipykernel_194024/3941071796.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [ ]:
chunk_size = 1024

folder_into_db_collection(
    '../data/KnowledgeStore',
    qdrant_client,
    CustomTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 2),
    sentence_embedder,
    prefix=prefixes['instance'])

In [7]:
retriever = CustomRetriever(
    qdrant_client=qdrant_client,
    embedder=sentence_embedder,
    collection_name=collection_name,
    prefix=prefixes['query'],
    )


In [9]:

search_result = retriever.get_relevant_documents(
    "Чем отличается REST от SOAP?", limit=4)
for result in search_result:
    pprint(result)
    print('****')
    print(result.payload['filename'])
    print(result.payload['text'])

print('===' * 10)

search_result = retriever.get_relevant_documents(
    "В чем суть CAP-теоремы?", limit=4)
for result in search_result:
    print('****')
    print(result.payload['filename'])
    print(result.payload['text'])

ScoredPoint(id='8009eecb-128a-41e6-bd5d-00de0389ee46', version=678, score=0.9016307, payload={'file_path': '../data/KnowledgeStore/REST.md', 'filename': 'REST.md', 'text': 'REST (REpresentational\xa0State\xa0Transfer) - \xa0архитектурный стиль \xa0взаимодействия компонентов распределённого приложения в\xa0сети.\nОн отличается от SOAP тем, что не является протоколом и стандартом.\n\n## Требования\n\n1. Модель клиент-сервер - разграничение на две стороны\n2. Stateless - вся необходимая информация находится в самом запросе/ответе\n3. Кэширование - реализуется на промежуточных серверах, необходимо для повышения производительности, сервер должен сам определить что кэшировать и как долго\n4. Унифицированный интерфейс - идентификация ресурсов по URI\n5. Уровневая структура - \n6. Код по требованию - сервер может передать код для клиента\n\n## Плохие варианты\n\n- Введение новых HTTP-методов\n- `/missiles/[id]?action=launch` - действия в параметре запроса\n- `/missiles/[id]/launch` - глагол в 